In [1]:
import requests
import cycquery.ops as qo
from cycquery import MIMICIVQuerier
import httpx
import json
import asyncio
import nest_asyncio
import re


def preprocess_medical_note(note):
    # Remove only excessive whitespace within lines, preserving newlines
    lines = note.split('\n')
    cleaned_lines = [' '.join(line.split()) for line in lines]
    return '\n'.join(cleaned_lines)
    

querier = MIMICIVQuerier(
    dbms="postgresql",
    port=5432,
    host="localhost",
    database="mimiciv-2.0",
    user="postgres",
    password="pwd",
)
# List all schemas.
querier.list_schemas()

2024-08-27 21:47:45,046 INFO cycquery.orm    - Database setup, ready to run queries!


['fhir_etl',
 'fhir_trm',
 'information_schema',
 'mimic_fhir',
 'mimiciv_derived',
 'mimiciv_ed',
 'mimiciv_hosp',
 'mimiciv_icu',
 'mimiciv_note',
 'public']

In [2]:
# List all tables.
querier.list_tables("mimiciv_note")
ops = qo.Sequential(qo.DropEmpty("text"), qo.DropNulls("text"))
notes = querier.mimiciv_note.discharge().ops(ops).run(limit=100)

2024-08-27 21:50:03,648 INFO cycquery.orm    - Query returned successfully!
2024-08-27 21:50:03,649 INFO cycquery.utils.profile - Finished executing function run_query in 0.355598 s


In [4]:
notes

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00,Name: ___ Unit No: __...
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00,Name: ___ Unit No: __...
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,Name: ___ Unit No: __...
3,10000032-DS-24,10000032,25742920,DS,24,2180-08-07 00:00:00,2180-08-10 05:43:00,Name: ___ Unit No: __...
4,10000084-DS-17,10000084,23052089,DS,17,2160-11-25 00:00:00,2160-11-25 15:09:00,Name: ___ Unit No: ___...
...,...,...,...,...,...,...,...,...
95,10002430-DS-7,10002430,26295318,DS,7,2129-06-24 00:00:00,2129-06-24 16:51:00,Name: ___ Unit No: ___ Adm...
96,10002443-DS-15,10002443,21329021,DS,15,2183-10-20 00:00:00,2183-10-20 19:18:00,Name: ___ Unit No: ...
97,10002495-DS-13,10002495,24982426,DS,13,2141-05-29 00:00:00,2141-05-30 02:29:00,Name: ___ Unit No: ___ ...
98,10002528-DS-17,10002528,23193578,DS,17,2168-12-20 00:00:00,2168-12-20 17:28:00,Name: ___ Unit No: ___ ...


In [ ]:
example_note = """
Patient Name: Jane Smith
Age: 58
Gender: Female
Date of Visit: August 25, 2024

Chief Complaint: Persistent headaches and dizziness for the past 3 weeks

History of Present Illness:
Jane Smith, a 58-year-old female, presents with a 3-week history of persistent headaches and dizziness. The headaches are described as throbbing, primarily located in the frontal and temporal regions, and rate 7/10 on a pain scale. They occur daily, lasting for several hours, and are partially relieved by over-the-counter pain medications. The patient reports associated symptoms of nausea, photophobia, and phonophobia. The dizziness is described as a sensation of the room spinning, exacerbated by sudden movements, and lasting for a few minutes at a time.

Past Medical History:
1. Hypertension (diagnosed 10 years ago)
2. Type 2 Diabetes Mellitus (diagnosed 5 years ago)
3. Hyperlipidemia

Medications:
1. Lisinopril 20mg daily
2. Metformin 1000mg twice daily
3. Atorvastatin 40mg daily
4. Aspirin 81mg daily

Allergies: Penicillin (rash)

Social History:
- Non-smoker
- Occasional alcohol use (1-2 glasses of wine per week)
- Works as a high school teacher
- Lives with husband, two children have moved out

Family History:
- Father: Myocardial infarction at age 65
- Mother: Stroke at age 70
- Sister: Migraine headaches

Review of Systems:
- General: Reports fatigue and decreased appetite
- HEENT: Denies vision changes, ear pain, or sinus congestion
- Cardiovascular: Denies chest pain or palpitations
- Respiratory: Denies shortness of breath or cough
- Gastrointestinal: Reports occasional nausea, denies vomiting or abdominal pain
- Musculoskeletal: Denies joint pain or muscle weakness
- Neurological: Reports headaches and dizziness as described above

Physical Examination:
- Vital Signs: BP 145/90, HR 78, RR 16, Temp 37.0°C, SpO2 98% on room air
- General: Alert and oriented, appears uncomfortable
- HEENT: PERRLA, EOM intact, no nystagmus, tympanic membranes clear bilaterally
- Cardiovascular: Regular rate and rhythm, no murmurs
- Respiratory: Clear to auscultation bilaterally
- Neurological: Cranial nerves II-XII intact, normal gait, negative Romberg test

Assessment and Plan:
1. Persistent headaches with associated symptoms suggestive of migraine
   - Order MRI brain to rule out intracranial pathology
   - Start sumatriptan 50mg as needed for acute attacks
   - Discuss lifestyle modifications and headache diary
2. Vertigo, possibly peripheral in origin
   - Refer to ENT for further evaluation
   - Consider vestibular rehabilitation therapy
3. Hypertension, currently uncontrolled
   - Increase lisinopril to 40mg daily
   - Encourage home blood pressure monitoring
   - Follow up in 2 weeks to reassess
4. Type 2 Diabetes Mellitus and Hyperlipidemia
   - Continue current management
   - Order HbA1c and lipid panel for next visit

Follow-up: Schedule follow-up appointment in 2 weeks to reassess headaches, dizziness, and blood pressure control. Advise patient to return sooner if symptoms worsen or new symptoms develop.
"""

In [ ]:
import httpx
import asyncio
from IPython import get_ipython


async def generate_text():
    url = 'http://localhost:8003/generate'
    input_prompt = 'I am trying to curate a careful list of questions and answers that can be used to train a LLM that can understand EHR data and answer questions based on the data. Use the medical note to create question answer pairs. For example: What medications do the patient take? Provide the output as Question: <question> \n Answer: <asnwer> format.'

    payload = {
        'prompt': input_prompt,
        'context': preprocess_medical_note(example_note)
    }

    timeout = httpx.Timeout(30.0, connect=60.0)
    async with httpx.AsyncClient(timeout=timeout) as client:
        try:
            async with client.stream('POST', url, json=payload) as response:
                if response.status_code == 200:
                    print("Response:")
                    full_response = ""
                    async for chunk in response.aiter_text():
                        print(chunk, end='', flush=True)
                        full_response += chunk
                    print("\n\nFull Response:", full_response)
                else:
                    print('Error:', response.status_code, await response.text())
        except httpx.ReadTimeout:
            print("The request timed out. The server is taking too long to respond.")
        except httpx.ConnectTimeout:
            print("Failed to connect to the server. Make sure the server is running and accessible.")
        except httpx.RequestError as exc:
            print(f"An error occurred while requesting {exc.request.url!r}.")
        except Exception as e:
            print(f"An unexpected error occurred: {str(e)}")

def run_async_code():
    ipython = get_ipython()
    if ipython is None:
        # We're not in IPython, use the default event loop
        asyncio.run(generate_text())
    else:
        # We're in IPython, use the IPython event loop
        import nest_asyncio
        nest_asyncio.apply()
        asyncio.get_event_loop().run_until_complete(generate_text())

# This allows the script to be imported without running the code
if __name__ == "__main__":
    run_async_code()